# Libraries and helpers

In [355]:
from polygon import StocksClient
import pandas as pd
import numpy as np
import Tiem
#Polygon api
api_key = "d04PM4tdE4SJ6cUpXG7TXv_nNzA1Bdp2"

# Testing

In [7]:
import datetime
#Function to get times
def get_unix_timestamps(day):
    # Set the time to 4:00AM
    time = datetime.time(hour=4)

    # Combine the date and time
    datetime_obj = datetime.datetime.combine(day, time)

    # Convert to a Unix timestamp
    unix_timestamp_4 = int(datetime_obj.timestamp())

    # Set the time to 9:00AM
    time = datetime.time(hour=9)

    # Combine the date and time
    datetime_obj = datetime.datetime.combine(day, time)

    # Convert to a Unix timestamp
    unix_timestamp_9 = int(datetime_obj.timestamp())

    return (unix_timestamp_4, unix_timestamp_9)

In [121]:
day = datetime.date(2023, 4, 17)
#start premarket hour
hour_4 = datetime.time(hour=4)
#start market hour
hour_9 = datetime.time(hour=9)
#closed time 
hour_16 = datetime.time(hour=16)

day_4 = datetime.datetime.combine(day,hour_4)
day_9 = datetime.datetime.combine(day,hour_9)
day_16 = datetime.datetime.combine(day,hour_16)


In [122]:
client = StocksClient(api_key, connect_timeout=10)
data = client.get_aggregate_bars('tsla',day_4, day_16,timespan='min')

In [112]:
results = data['results']
premarket = pd.DataFrame(results)
premarket.t = list(map(datetime.datetime.utcfromtimestamp,premarket.t/1000))

In [135]:
premarket.columns = ["volume","w_volumen","average","open","close","high","low","time","#transactions","otc_ticker"]

In [79]:
datetime.datetime.utcfromtimestamp(int(data['results'][-1]['t']/1000))

datetime.datetime(2023, 4, 17, 9, 0)

In [131]:
client = StocksClient(api_key, connect_timeout=10)
data = client.get_aggregate_bars('aapl',day_4, day_16,timespan='min')

In [133]:
results = data['results']
premarket1 = pd.DataFrame(results)
premarket1.t = list(map(datetime.datetime.utcfromtimestamp,premarket1.t/1000))
premarket1.columns = ["volume","w_volumen","average","open","close","high","low","time","#transactions","otc_ticker"]

In [146]:
day = datetime.date(2023, 4, 13)
data_day = client.get_daily_open_close('aapl',day)

In [ ]:
for i in data:
    if(i['status']=='OK'):
        print(i['symbol'],i['open'],i['close'],i['high'],i['low'],i['volume']))

# Getting all small caps companies symbols

In [381]:
import requests
from bs4 import BeautifulSoup

url = 'https://stockanalysis.com/list/small-cap-stocks/'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find_all('td')
#array of all symbols to be analysed
stocks = []
for i in range(len(table)):
    if (i-1)%7==0:
        stocks.append(table[i].text)
        
#number of companies to be analyzed
print(len(stocks))

# Downloading stock data (period = 1day) from 2021 to 2023

In [436]:
import yfinance as yf

data = yf.download(stocks,start=datetime.date(2021,1,1),end=datetime.date(2023,2,1),period="11",group_by="stock")

[*********************100%***********************]  500 of 500 completed

4 Failed downloads:
- AGM.A: No timezone found, symbol may be delisted
- AESI: Data doesn't exist for startDate = 1609477200, endDate = 1675227600
- HSAI: Data doesn't exist for startDate = 1609477200, endDate = 1675227600
- SYT: Data doesn't exist for startDate = 1609477200, endDate = 1675227600


In [443]:
a = set()
for i in list(data.columns):
    a.add(i[0])
    
#number of companies with data
print(len(a))

500


# Finding the Day1MOMO's stocks for each instance

In [868]:
#function to check premarket gaps bigger than 0.2
def day1MOMOs(data,stocks):
    aux = []
    #For all of our stocks
    for i in stocks:
        stock = data[i]
        for j in range(len(stock)-1):
            #Check the 
            premarket_gap= (stock.iloc[j+1].Open-stock.iloc[j].Close)/stock.iloc[j].Close
            if(premarket_gap>0.2 and datetime.datetime.date(stock.iloc[j+1].name) - datetime.datetime.date(stock.iloc[j].name)== datetime.timedelta(1)):
                aux.append([i,stock.iloc[j+1].name])
    return aux 
day1MOMOs = np.array(day1MOMOs(data,a))

# Downloading stock data (period = 1m) for each Day1MOMO where there was a gap in premarket 

In [507]:
#start premarket hour
hour_4 = datetime.time(hour=4)
#start market hour
hour_9 = datetime.time(hour=9)
#closed time 
hour_16 = datetime.time(hour=16)
data_market = []
for i in day1MOMOs[:2]:
    day = datetime.datetime.date(i[1])
    day_4 = datetime.datetime.combine(day,hour_4)
    day_9 = datetime.datetime.combine(day,hour_9)
    day_16 = datetime.datetime.combine(day,hour_16)
    aux = client.get_aggregate_bars(i[0],day_4,day_16,timespan='min')
    if aux['status']=='OK':
        results = aux['results']
        premarket = pd.DataFrame(results)
        premarket.t = list(map(datetime.datetime.utcfromtimestamp,premarket.t/1000))
        premarket.columns = ["volume","w_volumen","open","close","high","low","time","#transactions"]
        data_market.append([i[0],premarket])

# Divided the data in pre_market and market 

In [554]:
pre_markets = []
markets = []
d1 = data_market[1][1]
d1['time'] = pd.to_datetime(d1['time'])
d1[d1['time'].dt.hour<9]

for i in data_market:
    aux = i[1][pd.to_datetime(i[1]['time']).dt.hour<9]
    pre_markets.append((i[0],aux))
    aux = i[1][pd.to_datetime(i[1]['time']).dt.hour>=9]
    markets.append((i[0],aux))

# Getting red closes for Day1MOMOs

In [563]:
#get red closes
red_closes = []

for i in markets:
    red_closes.append((i[1].iloc[1,].open-i[1].iloc[-2,].close)/i[1].iloc[1,].open)
red_closes = np.array(red_closes)

In [569]:
#Get number of red closes
number_red_closes = len(red_closes[red_closes < 0])

1

# Fade and hour of premarket high

In [881]:
#% fade
premarket_fades = np.zeros([40])
premarket_high_hours = np.zeros([24])
for symbol,pre_market in markets:
    aux = pre_market[pre_market.high==max(pre_market.high)]
    time = aux.time
    high = aux.high
    close = aux.close
    premarket_fades[int(((high-close)/high).iloc[0] *10)]+=1
    premarket_high_hours[time.iloc[0].hour] +=1
    

# Candles 5,10,15

In [768]:
#Find candle types 

candles1  = {'red':0,'green':0}
candles5  = {'red':0,'green':0}
candles15  = {'red':0,'green':0}

for symbol,market in markets: 
    hour_candle_1 =   market.iloc[0].time+datetime.timedelta(minutes=1)
    hour_candle_5 =   market.iloc[0].time+datetime.timedelta(minutes=5)
    hour_candle_15 =   market.iloc[0].time+datetime.timedelta(minutes=15)
    
    candle_1 = market[market.time == hour_candle_1]
    candle_5 = market[market.time == hour_candle_5]
    candle_15 = market[market.time == hour_candle_15]
    
    if(len(candle_1)!=0 and candle_1.open.iloc[0]>candle_1.close.iloc[0]):
        candles1['red'] +=1 
    else: 
        candles1['green'] +=1 
        
    if(len(candle_5)!=0 and candle_5.open.iloc[0]>candle_5.close.iloc[0]):
        candles5['red'] +=1 
    else: 
        candles5['green'] +=1 
        
    if(len(candle_15)!=0 and candle_15.open.iloc[0]>candle_15.close.iloc[0]):
        candles15['red'] +=1 
    else: 
        candles15['green'] +=1 

# Fade percent 9:30 to 11:00

In [791]:
start_markets_fades = []

for symbol,market in markets:
    
    h_9 = pd.Timestamp.combine(market.iloc[0].time.date(),datetime.time(hour=9,minute=30))
    h_11 = pd.Timestamp.combine(market.iloc[0].time.date(),datetime.time(hour=11))
    
    aux = (market[market.time == h_9].open - market[market.time==h_11].open)/(market[market.time == h_9].open)

# Check if market exceed or touched high (High-Clearout)

In [951]:
highs = {}
for symbol,pre_market in markets:
    aux = pre_market[pre_market.high==max(pre_market.high)]
    print(aux)
    highs[symbol] = {'date':pre_market.iloc[0].time.date(),'high' : aux.high.iloc[0]}

   volume  w_volumen   open  close   high    low                time  \
0   17280    24.0318  15.33  23.11  44.29  15.33 2022-10-11 12:30:00   

   #transactions  
0            129  
     volume  w_volumen    open  close    high   low                time  \
173  480939    20.2618  20.065  20.44  20.625  20.0 2022-02-25 14:38:00   

     #transactions  
173           2219  


In [915]:
highs

{'FORG': {'date': datetime.date(2022, 10, 11), 'high': 44.29},
 'FTCH': {'date': datetime.date(2022, 2, 25), 'high': 20.625}}

In [837]:
high_clearout = {}
for symbol,market in markets:
    if(np.count_nonzero(market[['open','close','high']].min(axis='columns')>highs[symbol]['high'])==0):
        high_clearout[symbol] = {'market' : market} 

# Second Day Play - 2DayPlay

In [976]:
TwoDayPlay = []

for i in highs:
    aux = data[i][highs[i]['date'] + datetime.timedelta(days=1)== data[i].index].Open
    if(len(aux)!=0):
        if(abs(aux.iloc[0]-highs[i]['high'])>0.5):
            day = highs[i]['date'] + datetime.timedelta(days=1)
            day_4 = datetime.datetime.combine(day,hour_4)
            day_9 = datetime.datetime.combine(day,hour_9)
            day_16 = datetime.datetime.combine(day,hour_16)
            aux = client.get_aggregate_bars(i[0],day_4,day_16,timespan='min')
            if aux['status']=='OK':
                results = aux['results']
                premarket = pd.DataFrame(results)
                premarket.t = list(map(datetime.datetime.utcfromtimestamp,premarket.t/1000))
                premarket.columns = ["volume","w_volumen","open","close","high","low","time","#transactions"]
                TwoDayPlay.append([i,premarket])

/home/robdres/.local/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [977]:
TwoDayPlay

[['FORG',
         volume  w_volumen     open   close     high     low  \
  0       747.0    11.5700  11.5700  11.570  11.5700  11.570   
  1      2428.0    11.5700  11.5700  11.570  11.5700  11.570   
  2       580.0    11.6431  11.6400  11.640  11.6500  11.640   
  3       200.0    11.6315  11.6300  11.630  11.6300  11.630   
  4       289.0    11.6800  11.6800  11.680  11.6800  11.680   
  ..        ...        ...      ...     ...      ...     ...   
  270  302101.0    11.6212  11.6300  11.595  11.6400  11.595   
  271  209991.0    11.5984  11.6000  11.600  11.6100  11.590   
  272  242188.0    11.6036  11.5935  11.615  11.6150  11.590   
  273  137512.0    11.6118  11.6150  11.610  11.6200  11.600   
  274  240303.0    11.6159  11.6138  11.605  11.6276  11.600   
  
                     time  #transactions  
  0   2022-10-12 08:00:00              9  
  1   2022-10-12 08:04:00              5  
  2   2022-10-12 08:13:00              3  
  3   2022-10-12 08:26:00              2  
  4 